In [1]:
from comet_ml import Experiment
experiment = Experiment(api_key="xktj4EX0zB8YcQ3BEaFwOQYpu")


old comet version (1.0.6) detected. current: 1.0.8 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
Comet.ml support for Ipython Notebook is limited at the moment, automatic monitoring and stdout capturing is deactivated

Experiment is live on comet.ml https://www.comet.ml/incnas/general/13662c1d72254a63a91ca53f1edfb6e1



In [2]:
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras.regularizers import L1L2
from keras.utils import np_utils 

C:\Users\Duo\Anaconda3\envs\fastai\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import os
import fnmatch

import pandas as pd

## Load Data

In [4]:
train_path = "I:\\Honours-Project\\data\\sorted\\agg\\train\\"
train_csv = "I:\\Honours-Project\\data\\sorted\\agg\\train\\ids.csv"
test_path = "I:\\Honours-Project\\data\\sorted\\agg\\test\\"
test_csv = "I:\\Honours-Project\\data\\sorted\\agg\\test\\ids.csv"

dest_path = "I:\\Honours-Project\\data\\sorted\\agg\\"

In [5]:
x_train = []
y_train = []

x_test = []
y_test = []

'''for file in os.listdir(train_path):
    if fnmatch.fnmatch(file, '*.npy'):
        x_train.append(np.load(train_path+file))

x_train = np.array(x_train)    '''    

df = pd.read_csv(train_csv)

for path in df['name']:
    x_train.append(np.load(dest_path+path+".npy"))
    
x_train = np.array(x_train)
y_train = df["flow_rate"].values

'''for file in os.listdir(test_path):
    if fnmatch.fnmatch(file, '*.npy'):
        x_test.append(np.load(test_path+file))

x_test = np.array(x_test)        '''

df = pd.read_csv(test_csv)

for path in df['name']:
    x_test.append(np.load(dest_path+path+".npy"))
    
x_test = np.array(x_test)
y_test = df["flow_rate"].values

## Run Logistic Regression

In [6]:
batch_size = 128
nb_classes = 3
nb_epoch = 20

In [7]:
input_dim = x_train[0].shape[0] * x_train[0].shape[1]

In [8]:
x_train = x_train.reshape(x_train.shape[0], input_dim)
x_test = x_test.reshape(x_test.shape[0], input_dim)

In [9]:
x_train = x_train.astype('float32') 
x_test = x_test.astype('float32') 

# Might not be normalized to 255? Definitely not.
x_train /= 255 
x_test /= 255

In [10]:
y_train[y_train == 1800] = 0
y_train[y_train == 3600] = 1
y_train[y_train == 7200] = 2

y_test[y_test == 1800] = 0
y_test[y_test == 3600] = 1
y_test[y_test == 7200] = 2

y_train = np_utils.to_categorical(y_train, nb_classes) 
y_test = np_utils.to_categorical(y_test, nb_classes)

In [2]:
lmda = 0.01

def build_logistic_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(output_dim, input_dim=input_dim,
                    kernel_regularizer=regularizers.l2(lmda),
                    activation='softmax'))

    return model

In [12]:
model = build_logistic_model(input_dim, nb_classes)

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 1507314   
Total params: 1,507,314
Trainable params: 1,507,314
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [14]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=nb_epoch,
                    verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Train on 362 samples, validate on 40 samples
Epoch 1/20
362/362 [==============================] - 5s 15ms/step - loss: 1.0989 - acc: 0.3370 - val_loss: 1.0989 - val_acc: 0.3250
Epoch 2/20
362/362 [==============================] - 1s 2ms/step - loss: 1.0984 - acc: 0.3646 - val_loss: 1.0987 - val_acc: 0.3000
Epoch 3/20
362/362 [==============================] - 1s 2ms/step - loss: 1.0978 - acc: 0.3895 - val_loss: 1.0986 - val_acc: 0.3000
Epoch 4/20
362/362 [==============================] - 1s 2ms/step - loss: 1.0973 - acc: 0.4006 - val_loss: 1.0985 - val_acc: 0.3250
Epoch 5/20
362/362 [==============================] - 1s 2ms/step - loss: 1.0968 - acc: 0.4061 - val_loss: 1.0984 - val_acc: 0.3500
Epoch 6/20
362/362 [==============================] - 1s 2ms/step - loss: 1.0963 - acc: 0.4116 - val_loss: 1.0982 - val_acc: 0.3250
Epoch 7/20
362/362 [==============================] - 1s 2ms/step - loss: 1.0958 - acc: 0.4171 - val_loss: 1.0981 - val_acc: 0.3250
Epoch 8/20
362/362 [==========

In [15]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 1.0966670513153076
Test accuracy: 0.325
